# 빌트인 도구(built-in tools)

## 랭체인에서 제공하는 툴

1. Python REPL 도구

In [12]:
from dotenv import load_dotenv

load_dotenv()

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda
from langchain_experimental.tools import PythonREPLTool #pip install langchain_experimental

# 파이썬 코드를 실행하는 도구를 생성합니다.
python_tool = PythonREPLTool()

# 파이썬 코드를 실행하고 중간 과정을 출력하고 도구 실행 결과를 반환하는 함수
def print_and_execute(code, debug=True):
    if debug:
        print("CODE:")
        print(code)
    return python_tool.invoke(code)


# 파이썬 코드를 작성하도록 요청하는 프롬프트
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are Raymond Hetting, an expert python programmer, well versed in meta-programming and elegant, concise and short but well documented code. You follow the PEP8 style guide. "
            "Return only the code, no intro, no explanation, no chatty, no markdown, no code block, no nothing. Just the code.",
        ),
        ("human", "{input}"),
    ]
)
# LLM 모델 생성
llm = ChatOpenAI(model="gpt-4o", temperature=0)

# 프롬프트와 LLM 모델을 사용하여 체인 생성
chain = prompt | llm | StrOutputParser() | RunnableLambda(print_and_execute)

# 결과 출력
print(chain.invoke("로또 번호 생성기를 출력하는 코드를 작성하세요."))

CODE:
import random

def generate_lotto_numbers():
    return sorted(random.sample(range(1, 46), 6))

print(generate_lotto_numbers())
[3, 5, 8, 12, 18, 41]



2. TavilySearchResults : 검색 도구



In [2]:
from langchain_community.tools.tavily_search import TavilySearchResults

# 도구 생성
tool = TavilySearchResults(
    max_results=6,
    include_answer=True,
    include_raw_content=True,
    # include_images=True,
    # search_depth="advanced", # or "basic"
    include_domains=["github.io", "wikidocs.net"],
    # exclude_domains = []
)

# 도구 실행
tool.invoke({"query": "LangChain Tools 에 대해서 알려주세요"})

[{'url': 'https://wikidocs.net/262582',
  'content': '4 days ago · 도구(Tool)는 에이전트, 체인 또는 LLM이 외부 세계와 상호작용하기 위한 인터페이스입니다. LangChain 에서 기본 제공하는 도구를 사용하여 쉽게 도구를\xa0...'},
 {'url': 'https://teddylee777.github.io/langchain/langchain-agent/',
  'content': '태그:\nAgent,\nAPI KEY,\nFAISS,\nLangChain,\nLangSmith,\nmemory,\nOpenAI,\nPython,\nRetriever,\nTavily Search,\ntools,\n검색도구,\n랭체인,\n에이전트\n카테고리:\nlangchain\n업데이트: 2024년 02월 09일\n참고\nLangChain RAG 파헤치기: 다음의 추적 링크에서 자세한 단계별 수행 결과를 확인할 수 있습니다\nLangSmith 추적\n다음의 추적 링크에서 자세한 단계별 수행 결과를 확인할 수 있습니다\nLangSmith 추적\n④ 메모리 추가하기\n앞서 언급했듯이, 이 에이전트는 상태가 없습니다. LangChain 한국어 튜토리얼\n바로가기 👀\n[LangChain] 에이전트(Agent)와 도구(tools)를 활용한 지능형 검색 시스템 구축 가이드\n2024년 02월 09일\n41 분 소요\n이 글에서는 LangChain 의 Agent 프레임워크를 활용하여 복잡한 검색과 📍 전체 템플릿 코드\n다음의 추적 링크에서 자세한 단계별 수행 결과를 확인할 수 있습니다\nLangSmith 추적\n마무리입니다!\n 문서 기반 QA 시스템 설계 방법 - 심화편\n2024년 02월 06일\n22 분 소요\nLangChain의 RAG 시스템을 통해 문서(PDF, txt, 웹페이지 등)에 대한 질문-답변을 찾는 과정을 정리하였습니다.\n'},
 {'url': 'https://wikidocs.net/233351',
  'co

# 사용자 정의 도구(Custom Tool)

In [16]:
from langchain.tools import tool


# 데코레이터를 사용하여 함수를 도구로 변환합니다.
@tool
def add_numbers(a: int, b: int) -> int:
    """Add two numbers"""
    return a + b


@tool
def multiply_numbers(a: int, b: int) -> int:
    """Multiply two numbers"""
    return a * b

print(add_numbers.invoke({"a": 3, "b": 4}))

print(multiply_numbers.invoke({"a": 3, "b": 4}))

7
12


## 구글 뉴스기사 검색 도구

In [18]:
from langchain_teddynote.tools import GoogleNews

# 도구 생성
news_tool = GoogleNews()

# 최신 뉴스 검색
news_tool.search_by_keyword("AI 투자", k=5)
#news_tool.search_by_keyword(k=5)

[{'url': 'https://news.google.com/rss/articles/CBMiakFVX3lxTE81QVlXam1wclZOV3loQndWUjVjU3FKUE9zMzRjbkU5b3RMSjFBaGVfMHBvM28wRDFyd05yMVhQYTVnZy1nVkhGZXBqLTV2Zks5bnFQdG8wVEtBeEJoZllhVkQwQ2d1MENteEE?oc=5',
  'content': '오픈AI, 투자 유치 이후 글로벌 확장...뉴욕·파리·싱가포르 등 5곳에 지사 개설 - AI타임스'},
 {'url': 'https://news.google.com/rss/articles/CBMiW0FVX3lxTE4yZEVnMVBTeGNZWEZzQmpzeHhsZ2o5UTVSR3g2OXhNUTZLendsT2R1Zlc1ejUzd3VLaWk1LXhnOElyY3lRSkRfYm5aUGIzXzRja0lSRE5QUFhzUmfSAV5BVV95cUxOSXJ5YU1xU1lrYmpJQWZVRXoycHhxaVp2eWVNb0Q5aW1kTWtTZzlBdXVsaXpiUm5OUXNpMFB1WXMyWWd6c28tVHFzUXgyMVNYaExRQWRKRzZ0bU5UQXRR?oc=5',
  'content': 'AI 검색 라이너, 270억원 시리즈B 투자 유치 - 연합뉴스'},
 {'url': 'https://news.google.com/rss/articles/CBMiWkFVX3lxTE41Uk9WUHVRS196S1ByVWx1V1IxQnlQbHhMSE03RXA5NXQ2cWk1RmMzX3kweVpQeEw1OWo5TzM3NGtWT2tMMW1NdXlEV2hIRmg2bVQ0aEROWllxQdIBVEFVX3lxTE04dDMyOE80dUNSNFMycFpmTTQ1TGUtQjREZlFGYXZpM2tuU3huQnVzWElRSUZFbEJFZlhBOTZFY3VLd2ZpRXVIVFp5YTY2aDBYTi1hRQ?oc=5',
  'content': 'KB증권, 생성 AI 활용해 양방향 맞춤형 투자정보 제공 - 한국경제'},
 {'url': '

구글 뉴스 검색 툴

In [19]:
from langchain_teddynote.tools import GoogleNews
from langchain.tools import tool
from typing import List, Dict


# 키워드로 뉴스 검색하는 도구 생성
@tool
def search_keyword(query: str) -> List[Dict[str, str]]:
    """Look up news by keyword"""
    print(query)
    news_tool = GoogleNews()
    return news_tool.search_by_keyword(query, k=5)

# 실행 결과
search_keyword.invoke({"query": "AI 투자"})

AI 투자


[{'url': 'https://news.google.com/rss/articles/CBMiakFVX3lxTE81QVlXam1wclZOV3loQndWUjVjU3FKUE9zMzRjbkU5b3RMSjFBaGVfMHBvM28wRDFyd05yMVhQYTVnZy1nVkhGZXBqLTV2Zks5bnFQdG8wVEtBeEJoZllhVkQwQ2d1MENteEE?oc=5',
  'content': '오픈AI, 투자 유치 이후 글로벌 확장...뉴욕·파리·싱가포르 등 5곳에 지사 개설 - AI타임스'},
 {'url': 'https://news.google.com/rss/articles/CBMiWkFVX3lxTE41Uk9WUHVRS196S1ByVWx1V1IxQnlQbHhMSE03RXA5NXQ2cWk1RmMzX3kweVpQeEw1OWo5TzM3NGtWT2tMMW1NdXlEV2hIRmg2bVQ0aEROWllxQdIBVEFVX3lxTE04dDMyOE80dUNSNFMycFpmTTQ1TGUtQjREZlFGYXZpM2tuU3huQnVzWElRSUZFbEJFZlhBOTZFY3VLd2ZpRXVIVFp5YTY2aDBYTi1hRQ?oc=5',
  'content': 'KB증권, 생성 AI 활용해 양방향 맞춤형 투자정보 제공 - 한국경제'},
 {'url': 'https://news.google.com/rss/articles/CBMiW0FVX3lxTE4yZEVnMVBTeGNZWEZzQmpzeHhsZ2o5UTVSR3g2OXhNUTZLendsT2R1Zlc1ejUzd3VLaWk1LXhnOElyY3lRSkRfYm5aUGIzXzRja0lSRE5QUFhzUmfSAV5BVV95cUxOSXJ5YU1xU1lrYmpJQWZVRXoycHhxaVp2eWVNb0Q5aW1kTWtTZzlBdXVsaXpiUm5OUXNpMFB1WXMyWWd6c28tVHFzUXgyMVNYaExRQWRKRzZ0bU5UQXRR?oc=5',
  'content': 'AI 검색 라이너, 270억원 시리즈B 투자 유치 - 연합뉴스'},
 {'url': '